
## **A program For Photographers Event**
# Introduction

ABC is a company based in Toronto for organizing events, currently it works on a project to organize an event for 5 days for a group of photographers from all over the world.The company has to put a good program, including a hotel of residence, a hall for meetings, places of landscape to visit, stores for shopping, restaurants and cafes. So the company’s purpose is to make a list of places of landscape in Toronto, including the nearest restaurants, cafes, and shopping stores for each place.
Data Description

The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.

Table of contents

1- Import Libraries

2- Define Foursquare Credentials

3- Search for Hotels

### Import Libraries


In [0]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

### DEFINE FOURSQUARE CREDENTIALS

In [0]:
ClIENT_ID = '4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0' # your Foursquare ID
ClIENT_SECRET = 'RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: 4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0
Foursquare_Secret:RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ


In [0]:
# define the city and get its latitude & longitude 
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [0]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Hotel&radius=500&limit=30'

In [0]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [0]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,51d212c3498ebf27dc469bc9,Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587220886,False,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,NaN
1,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587220886,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
2,4af96fbbf964a520c01122e3,One King West Hotel & Residence,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587220886,False,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,NaN
3,4ae61cf6f964a520caa421e3,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587220886,False,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,NaN
4,51e48697498eded9073c6c17,Crew Room Eaton Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587220886,False,33 Gerrard Street West,NaN,43.658094,-79.382711,"[{'label': 'display', 'lat': 43.65809435580944...",522,NaN,CA,Toronto,ON,Canada,"[33 Gerrard Street West, Toronto ON, Canada]",NaN,NaN


In [0]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Chelsea Hotel,Hotel,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,51d212c3498ebf27dc469bc9
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
2,One King West Hotel & Residence,Hotel,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,4af96fbbf964a520c01122e3
3,Pantages Hotel & Spa,Hotel,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,4ae61cf6f964a520caa421e3
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,NaN,43.658094,-79.382711,"[{'label': 'display', 'lat': 43.65809435580944...",522,NaN,CA,Toronto,ON,Canada,"[33 Gerrard Street West, Toronto ON, Canada]",NaN,51e48697498eded9073c6c17


In [0]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
5,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
8,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON
9,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON


In [0]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

	

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
5,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
13,Pantages Hotel Gym,Gym,200 Victoria St.,43.654464,-79.379036,M5B 1V8,ON
14,Pantages Hotel Toronto Centre,Residential Building (Apartment / Condo),200 Victoria Street,43.655205,-79.379182,M5B 1V8,ON
16,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
18,The Spa At The Adelaide Hotel,Spa,325 Bay St,43.649725,-79.380346,M5H 4G5,ON


In [0]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
16,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
19,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
25,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
27,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
29,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [0]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
16,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
19,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
25,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
27,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
29,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [0]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
1,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
29,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


In [0]:


# search for Parks
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Park&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
presults = requests.get(url).json()
#presults

# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1587221052,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",NaN
1,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1587221052,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN
2,4b9177e4f964a520d2be33e3,TTC Streetcar #403 - Victoria park,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",v-1587221052,False,NaN,NaN,43.650735,-79.386921,"[{'label': 'display', 'lat': 43.650735, 'lng':...",389,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN
3,4d3a1d23325ff04db2dd1845,Residences at College Park North,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1587221052,False,763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",712,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,NaN
4,4bfc5019da7120a132df49fd,St. Andrews Playground / Dog Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1587221052,False,450 Adelaide Street West,Brant St & Adelaide St W,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,NaN,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,NaN


In [0]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,M5G 2P1,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,NaN,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",4b54deadf964a520a3d027e3
2,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,NaN,43.650735,-79.386921,"[{'label': 'display', 'lat': 43.650735, 'lng':...",389,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",4b9177e4f964a520d2be33e3
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,btwn. College St. and Gerrard St.,43.659822,-79.385159,"[{'label': 'display', 'lat': 43.65982157793473...",712,M5G 2R3,CA,Toronto,ON,Canada,[763 Bay St. (btwn. College St. and Gerrard St...,4d3a1d23325ff04db2dd1845
4,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,Brant St & Adelaide St W,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,NaN,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,4bfc5019da7120a132df49fd


In [0]:
# delete unnecessary columns
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
4,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
5,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
7,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
8,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
9,Moss Park Tennis Courts,Tennis Court,Moss Park,43.655337,-79.371923,NaN,ON


In [0]:


# delete rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3



,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
3,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
5,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
7,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
8,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
14,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
17,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
18,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
19,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON


In [0]:


# delete rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park



,name,categories,address,lat,lng,postalCode,state
6,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
14,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
17,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
18,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
19,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
22,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
23,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
25,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
29,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON


In [0]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Restaurant&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1587221153,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1587221153,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
2,4b266f05f964a520657b24e3,Goldstone Noodle Restaurant 金石,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1587221153,False,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",1143,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,NaN
3,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1587221153,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN
4,4b072e9df964a52009f922e3,Sky Dragon Chinese Restaurant 龍翔酒樓,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1587221153,False,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",1149,NaN,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,NaN


In [0]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
2,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,"[{'label': 'display', 'lat': 43.6522783893466,...",1143,M5T 2E4,CA,Toronto,ON,Canada,"[266 Spadina Ave (at Willison Sq), Toronto ON ...",at Willison Sq,NaN,4b266f05f964a520657b24e3
3,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3
4,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,"[{'label': 'display', 'lat': 43.65278331265585...",1149,NaN,CA,Toronto,ON,Canada,"[280 Spadina Ave. (at Dundas St. W.), Toronto ...",at Dundas St. W.,NaN,4b072e9df964a52009f922e3


In [0]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
3,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
4,Sky Dragon Chinese Restaurant 龍翔酒樓,Dim Sum Restaurant,280 Spadina Ave.,43.652783,-79.398174,NaN,ON
5,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
6,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
7,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
8,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
9,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON


In [0]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,Goldstone Noodle Restaurant 金石,Noodle House,266 Spadina Ave,43.652278,-79.398039,M5T 2E4,ON
3,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
5,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
7,Swatow Restaurant 汕頭小食家,Chinese Restaurant,309 Spadina Ave.,43.653866,-79.398334,M5T 2E6,ON
8,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
9,360 Restaurant,Wine Bar,301 Front St W,43.642537,-79.387042,M5V 2T6,ON
10,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
12,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON


In [0]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [0]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f9165bde4b04fef0087e4e2,Cafeteria,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1587221218,False,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN
1,528e99a211d262edc3708a6f,Stay Cafeteria 慢走,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1587221218,False,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",1246,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5
2,4b57256bf964a520332828e3,Sears Cafeteria,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1587221218,False,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN
3,53bd6dfe498e6c1d3bc9333e,Industrial Alliance Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1587221218,False,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
4,4ec2b68a8231a83de8c45191,OCAD Cafeteria,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",v-1587221218,False,100 McCaul St.,43.652780,-79.391762,"[{'label': 'display', 'lat': 43.65278016892648...",635,CA,Toronto,ON,Canada,"[100 McCaul St. (Second Floor), Toronto ON, Ca...",Second Floor,NaN


In [0]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,"[{'label': 'display', 'lat': 43.65545427446646...",1246,CA,Toronto,ON,Canada,"[388 Spadina Ave (at Nassau St), Toronto ON M5...",at Nassau St,M5T 2G5,528e99a211d262edc3708a6f
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN,4b57256bf964a520332828e3
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,53bd6dfe498e6c1d3bc9333e
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,"[{'label': 'display', 'lat': 43.65278016892648...",635,CA,Toronto,ON,Canada,"[100 McCaul St. (Second Floor), Toronto ON, Ca...",Second Floor,NaN,4ec2b68a8231a83de8c45191


In [0]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,state,postalCode
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,ON,NaN
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,ON,NaN
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,ON,NaN
4,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,ON,NaN
5,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,ON,NaN
6,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
7,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,ON,NaN
8,SickKids Cafeteria,Food Court,555 University Avenue,43.657584,-79.375652,ON,NaN
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7


In [0]:


# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria



,name,categories,address,lat,lng,state,postalCode
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,ON,M5T 2G5
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
18,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,ON,M5B 1W8
21,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,ON,M5B 2K3
28,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,ON,m3c 3k7
29,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,ON,M5S 1J5


In [0]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4FQQJE2LMGVLERRRE2BOFR1LQQR5ZEHFLTD1WRAFKOFT4TI0&client_secret=RCSNR5TH31NQRG2QYCHK101HRMTUZHBB2TSJ1CJU5UOGRXRQ&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=30'

In [0]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1587221329,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
1,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1587221329,False,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"
2,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1587221329,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN


In [0]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN,5840cafe06f1a34af80cc609
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN,4b9e86dff964a520e6eb36e3


In [0]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [0]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [0]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON
2,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
3,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
4,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
5,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
6,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON
7,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,M5S 1J5,ON
8,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON
9,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [0]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

In [0]:
# create dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
1,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
3,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
4,Barbara Hall Park,Park,519 Church St,43.666879,-79.381068,M4Y 2K9,ON
5,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
6,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
7,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,M5V 3Z1,ON
8,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
9,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON


In [0]:
# Generate map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map

This is the link to Google Collab to see maps (Wont't be visible in Jupyter Notebook):


https://colab.research.google.com/drive/1eklEN9Dq8EEyIHqulpewyKrtdfNq24_Y